# LAB | Abstractive Question Answering

Abstractive question-answering focuses on the generation of multi-sentence answers to open-ended questions. It usually works by searching massive document stores for relevant information and then using this information to synthetically generate answers. This notebook demonstrates how Pinecone helps you build an abstractive question-answering system. We need three main components:

- A vector index to store and run semantic search
- A retriever model for embedding context passages
- A generator model to generate answers

# Install Dependencies

In [47]:
#!pip install -qU datasets pinecone-client==3.1.0 sentence-transformers torch

# Load and Prepare Dataset

Our source data will be taken from the Wiki Snippets dataset, which contains over 17 million passages from Wikipedia. But, since indexing the entire dataset may take some time, we will only utilize 50,000 passages in this demo that include "History" in the "section title" column. If you want, you may utilize the complete dataset. Pinecone vector database can effortlessly manage millions of documents for you.

In [48]:
from datasets import load_dataset

# load the dataset from huggingface in streaming mode and shuffle it
wiki_data = load_dataset(
    'vblagoje/wikipedia_snippets_streamed',
    split='train',
    streaming=True
).shuffle(seed=960)

type(wiki_data)

datasets.iterable_dataset.IterableDataset

We are loading the dataset in the streaming mode so that we don't have to wait for the whole dataset to download (which is over 9GB). Instead, we iteratively download records one at a time.

In [49]:
# show the contents of a single document in the dataset
record = next(iter(wiki_data))
record

{'wiki_id': 'Q7649565',
 'start_paragraph': 20,
 'start_character': 272,
 'end_paragraph': 24,
 'end_character': 380,
 'article_title': 'Sustainable Agriculture Research and Education',
 'section_title': "2000s & Evaluation of the program's effectiveness",
 'passage_text': "preserving the surrounding prairies. It ran until March 31, 2001.\nIn 2008, SARE celebrated its 20th anniversary. To that date, the program had funded 3,700 projects and was operating with an annual budget of approximately $19 million. Evaluation of the program's effectiveness As of 2008, 64% of farmers who had received SARE grants stated that they had been able to earn increased profits as a result of the funding they received and utilization of sustainable agriculture methods. Additionally, 79% of grantees said that they had experienced a significant improvement in soil quality though the environmentally friendly, sustainable methods that they were"}

In [50]:
# filter only documents with History as section_title - Replace None with your code
history = (record for record in wiki_data if record['section_title'] == 'History')

for i, record in enumerate(history):
    if i == 3:
        break
    print(record)
    print()

{'wiki_id': 'Q2644349', 'start_paragraph': 10, 'start_character': 397, 'end_paragraph': 10, 'end_character': 534, 'article_title': 'Taupo District', 'section_title': 'History', 'passage_text': 'was not until the 1950s that the region started to develop, with forestry and the construction of the Wairakei geothermal power station.'}

{'wiki_id': 'Q7718184', 'start_paragraph': 2, 'start_character': 0, 'end_paragraph': 6, 'end_character': 553, 'article_title': 'The Bishop Wand Church of England School', 'section_title': 'History', 'passage_text': "The Bishop Wand Church of England School History The Bishop Wand Church of England School was founded in 1969 to serve, alongside non-Anglican schools and a few fellow Anglican-ethos schools, the borough of Spelthorne and the London Boroughs of Richmond upon Thames and Hounslow.\nThe prominent Anglican cleric for whom the school is named is Bishop William Wand who served 22 years as a Bishop then resigned in 1956 to serve as Canon and Treasurer o

Let's iterate through the dataset and apply our filter to select the 50,000 historical passages. We will extract `article_title`, `section_title` and `passage_text` from each document.

In [51]:
from tqdm.auto import tqdm  # progress bar

# total_doc_count = 50000
total_doc_count = 100

counter = 0
docs = []
# iterate through the dataset and apply our filter
for d in tqdm(history, total=total_doc_count):
    # extract the fields we need - article, section, and passage
    doc = {
        "article_title": d["article_title"],
        "section_title": d["section_title"],
        "passage_text": d["passage_text"]
    }
    docs.append(doc)
    # increase the counter on every iteration
    counter += 1
    if counter >= total_doc_count:
        break

  0%|          | 0/100 [00:00<?, ?it/s]

In [52]:
import pandas as pd

# create a pandas dataframe with the documents we extracted
df = pd.DataFrame(docs)
df.head()

,article_title,section_title,passage_text
0,Swanton House,History,it. Lane provided funds for restoration by the...
1,Takashinohama Line,History,Takashinohama Line The Takashinohama Line (高師浜...
2,Tamil Methodist Church,History,Tamil Methodist Church History The church was ...
3,Star Music,History,in order to strengthen its production base and...
4,Terai,History,timber reserves.\nIndian immigration increased...


# Initialize Pinecone Index

The Pinecone index stores vector representations of our historical passages which we can retrieve later using another vector (query vector). To build our vector index, we must first establish a connection with Pinecone. For this, we need an API from Pinecone. You can get one for free from [here](https://app.pinecone.io/), and after that, we initialize the connection as follows:

In [53]:
import os
from pinecone import Pinecone
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

PINECONE_API_KEY= os.getenv('PINECONE_API_KEY')
# initialize connection to pinecone (get API key at app.pinecone.io)
api_key = os.environ.get('PINECONE_API_KEY') or 'PINECONE_API_KEY'

# configure client
pc = Pinecone(api_key=api_key)
pc

Now we setup our index specification, this allows us to define the cloud provider and region where we want to deploy our index. You can find a list of all [available providers and regions here](https://docs.pinecone.io/docs/projects).

In [54]:
from pinecone import ServerlessSpec

cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'

spec = ServerlessSpec(cloud=cloud, region=region)

Now we create a new index. We will name it "abstractive-question-answering" — you can name it anything we want. We specify the metric type as "cosine" and dimension as 768 because the retriever we use to generate context embeddings is optimized for cosine similarity and outputs 768-dimension vectors.

In [55]:
index_name = "abstractive-question-answering" 

# Check if index exists, if not, create a new one
if index_name not in pc.list_indexes():
    pc.create_index(index_name, dimension=768, metric='cosine', spec=spec)

In [56]:
import time

# Initialize the index (if not already initialized)
index = pc.Index(index_name)

# Wait for a moment to ensure the index is fully initialized
time.sleep(2)

# Retrieve and check the index stats
stats = index.describe_index_stats()

# Ensure the number of vectors is zero
if stats['total_vector_count'] == 0:
    print("Index initialized successfully, with 0 vectors.")
else:
    print(f"Index already contains {stats['total_vector_count']} vectors.")


Index initialized successfully, with 0 vectors.


# Initialize Retriever

Next, we need to initialize our retriever. The retriever will mainly do two things:

- Generate embeddings for all historical passages (context vectors/embeddings)
- Generate embeddings for our questions (query vector/embedding)

The retriever will create embeddings such that the questions and passages that hold the answers to our queries are close to one another in the vector space. We will use a SentenceTransformer model based on Microsoft's MPNet as our retriever. This model performs quite well for comparing the similarity between queries and documents. We can use Cosine Similarity to compute the similarity between query and context vectors generated by this model (Pinecone automatically does this for us).

In [57]:
import torch
from sentence_transformers import SentenceTransformer

# set device to GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# load the retriever model from huggingface model hub
retriever = SentenceTransformer('flax-sentence-embeddings/all_datasets_v3_mpnet-base') 
retriever

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

# Generate Embeddings and Upsert

Next, we need to generate embeddings for the context passages. We will do this in batches to help us more quickly generate embeddings and upload them to the Pinecone index. When passing the documents to Pinecone, we need an id (a unique value), context embedding, and metadata for each document representing context passages in the dataset. The metadata is a dictionary containing data relevant to our embeddings, such as the article title, section title, passage text, etc.

In [58]:
# we will use batches of 64
batch_size = 64

#You will create embedding for the passage_text variable and be use to include the meta data in each batch
for i in tqdm(range(0, len(df), batch_size)):
    # find end of batch
    end_of_batch = i + batch_size
    # extract batch
    batch = df.iloc[i:end_of_batch]
    # generate embeddings for batch
    emb = retriever.encode(batch["passage_text"].tolist())
    # prepare metadata for each passage
    meta_title = batch['article_title']
    meta_section = batch['section_title']
    meta_passage = batch['passage_text']
    # create unique ids
    ids = [str(idx) for idx in range(i, i + len(batch))]
    # add all to upsert list
    to_upsert = [
        (ids[j], emb[j], {
            'article_title': meta_title.iloc[j],
            'section_title': meta_section.iloc[j],
            'passage_text': meta_passage.iloc[j]
        })
        for j in range(len(batch))
    ]
    # upsert all to pinecone
    index.upsert(vectors=to_upsert)

# check that we have all vectors in index
index.describe_index_stats()



  0%|          | 0/2 [00:00<?, ?it/s]

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 64}},
 'total_vector_count': 64}

# Initialize Generator

We will use ELI5 BART for the generator which is a Sequence-To-Sequence model trained using the ‘Explain Like I’m 5’ (ELI5) dataset. Sequence-To-Sequence models can take a text sequence as input and produce a different text sequence as output.

The input to the ELI5 BART model is a single string which is a concatenation of the query and the relevant documents providing the context for the answer. The documents are separated by a special token &lt;P>, so the input string will look as follows:

>question: What is a sonic boom? context: &lt;P> A sonic boom is a sound associated with shock waves created when an object travels through the air faster than the speed of sound. &lt;P> Sonic booms generate enormous amounts of sound energy, sounding similar to an explosion or a thunderclap to the human ear. &lt;P> Sonic booms due to large supersonic aircraft can be particularly loud and startling, tend to awaken people, and may cause minor damage to some structures. This led to prohibition of routine supersonic flight overland.

More detail on how the ELI5 dataset was built is available [here](https://arxiv.org/abs/1907.09190) and how ELI5 BART model was trained is available [here](https://yjernite.github.io/lfqa.html).

Let's initialize the BART model using transformers.

In [59]:
from transformers import BartTokenizer, BartForConditionalGeneration

# load bart tokenizer and model from huggingface
tokenizer = BartTokenizer.from_pretrained('vblagoje/bart_lfqa')
generator = BartForConditionalGeneration.from_pretrained('vblagoje/bart_lfqa').to(device)

All the components of our abstract QA system are complete and ready to be queried. But first, let's write some helper functions to retrieve context passages from Pinecone index and to format the query in the way the generator expects the input.

In [60]:
def query_pinecone(query, top_k):
    # generate embeddings for the query
    xq = retriever.encode(query).tolist()
    # search pinecone index for context passage with the answer
    xc = index.query(
        vector=xq,
        top_k=top_k,
        include_values=True,
        include_metadata=True,
    )
    return xc

In [61]:
def format_query(query, context):
    # extract passage_text from Pinecone search result and add the <P> tag
    context = [f"<P> {m['metadata']['passage_text']}" for m in context]
    # concatinate all context passages
    context = " ".join(context)
    # contcatinate the query and context passages
    query = f"question: {query} context: {context}"
    return query

Let's test the helper functions. We will query the Pinecone index function we created earlier with the `query_pinecone` to get context passages and pass them to the `format_query` function.

In [62]:
# query = "when was the first electric power system built?"
query = "Who discovered the rosetta stone?"
result = query_pinecone(query, top_k=1)
result

{'matches': [{'id': '90',
              'metadata': {'article_title': "Tofo-Sant'Eleuterio",
                           'passage_text': 'Felice.\n'
                                           'In the 1960s the village began to '
                                           'industrialize more heavily.',
                           'section_title': 'History'},
              'score': 0.20781225,
              'values': [-0.072517015,
                         0.0254869889,
                         0.00312633207,
                         0.00460676942,
                         0.0501532964,
                         -0.0417916924,
                         0.0158646,
                         0.00572287571,
                         0.0633073151,
                         0.085800007,
                         0.0561944358,
                         0.0616545044,
                         0.0155155426,
                         -0.0190654788,
                         -0.0346193537,
                    

In [63]:
from pprint import pprint

In [64]:
# format the query in the form generator expects the input
query = format_query(query, result["matches"])
pprint(query)

('question: Who discovered the rosetta stone? context: <P> Felice.\n'
 'In the 1960s the village began to industrialize more heavily.')


The output looks great. Now let's write a function to generate answers.

In [65]:
def generate_answer(query):
    # tokenize the query to get input_ids
    inputs = tokenizer([query], max_length=1024, return_tensors="pt").to(device)
    # use generator to predict output ids
    ids = generator.generate(inputs["input_ids"], num_beams=2, min_length=20, max_length=40)
    # use tokenizer to decode the output ids
    answer = tokenizer.batch_decode(ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    return pprint(answer)

In [66]:
generate_answer(query)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


('The rosetta stone was discovered in the late 19th century in the town of '
 'Toulouse, France. It was discovered by a Frenchman named Jean-Baptiste '
 'Gagnon')


As we can see, the generator used the provided context to answer our question. Let's run some more queries.

In [67]:
# query = "How was the first wireless message sent?"
query = "What are the different components of wine?"
context = query_pinecone(query, top_k=5)
query = format_query(query, context["matches"])
generate_answer(query)

('Wine is made up of grapes. The grapes are used to make wine, and the grapes '
 'are used to make the wine. Wine is made up of a lot of different things, but')


To confirm that this answer is correct, we can check the contexts used to generate the answer.

In [68]:
for doc in context["matches"]:
    print(doc["metadata"]["passage_text"], end='\n---\n')

Felice.
In the 1960s the village began to industrialize more heavily.
---
The Siena School History The Siena School was founded in 2006, serving students in grades 5-9. They were originally located at the Montgomery Hills Baptist Church along Georgia Avenue in Silver Spring, MD. The school expanded to serve students in grades 4-12, and moved into its permanent location on Forest Glen Road in 2013. Siena is accredited by the Middle States Association of Colleges and Schools Commissions on Elementary and Secondary Schools (MSA-CESS). The school’s namesake is Siena, Italy for two reasons: (1) this Tuscan city is well-known for its art, a theme which Siena incorporates into its day-to-day curriculum,
---
built to act simultaneously as canals for boats. At that time massive inroads were made into the environment and millions of cubic metres of peat were cut. The peat was sold for heating fuel and shipped to Bremen using peat barges. The embankments running alongside these canals were used b

In this case, the answer looks correct. If we ask a question and no relevant contexts are retrieved, the generator will typically return nonsensical or false answers, like with this question about COVID-19:

In [69]:
query = "where did COVID-19 originate?"
context = query_pinecone(query, top_k=3)
query = format_query(query, context["matches"])
generate_answer(query)

('COVID-19 is a name for the COVID-19 program, which was a program that was '
 "developed by the United States Army in response to the Soviet Union's "
 'invasion of Afghanistan.')


In [70]:
for doc in context["matches"]:
    print(doc["metadata"]["passage_text"], end='\n---\n')

the transfer of the Italian Consulate and installation of the Italian Institute of Culture in the building the theater.
---
Felice.
In the 1960s the village began to industrialize more heavily.
---
Major 'melioration' measures such as draining, deep ploughing (Tiefumbruch) and river regulation were supposed to increase the productivity of agriculture and even enabled arable farming. Intensive farming methods were used to grow maize as an animal feedstuff. These measures had been supported since the middle of the 20th century by various national and European subsidy programmes. This went so far that ditches dried out in summer, heath fires broke out and, during sustained periods of drought, the land was artificially watered.
In the 1990s a major rethink began. By leaving the land to regenerate and by reflooding it, attempts have been
---


Let’s finish with a final few questions.

In [71]:
query = "what was the war of currents?"
context = query_pinecone(query, top_k=5)
query = format_query(query, context["matches"])
generate_answer(query)

('The War of Currents was a series of naval battles between the British and '
 'French navies during the Napoleonic Wars. The British and French navies '
 'fought each other in the Battle of')


In [72]:
query = "who was the first person on the moon?"
context = query_pinecone(query, top_k=10)
query = format_query(query, context["matches"])
generate_answer(query)

('The first person to walk on the moon was Neil Armstrong, who walked on the '
 'moon in 1969.')


In [73]:
query = "what was NASAs most expensive project?"
context = query_pinecone(query, top_k=3)
query = format_query(query, context["matches"])
generate_answer(query)

('The Space Shuttle was the most expensive project in the history of the '
 'United States. It cost about $100 billion to build.')


As we can see, the model can generate some decent answers.

#### Add a few more questions

In [75]:
query = "Who killed Kennedy?"
context = query_pinecone(query, top_k=3)
query = format_query(query, context["matches"])
generate_answer(query)


("I'm going to assume you're talking about the assassination of JFK. The "
 'assassination of JFK was carried out by a group of people who were known to '
 'the US government as the Weathermen.')


In [76]:
query = "Where can I buy beer?"
context = query_pinecone(query, top_k=3)
query = format_query(query, context["matches"])
generate_answer(query)


("I'm not sure about the rest of the world, but in the UK, you can buy beer "
 'from a brewery in the city of London.')
